In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as tgrad
import torch.nn.functional as F

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from timeit import default_timer as timer

import importlib
import utils
import networks

# change the default figure size
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
plt.rcParams.update({'font.size': 16})

In [ ]:
os.environ['KMP_DUPLICATE_LIB_OK']='True'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(torch.cuda.is_available())
print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name())

In [ ]:
# model parameters
K = 10
r = 0.035
sigma = 0.2
T = 1
S_range = [0, int(5*K)]
t_range = [0, T]
gs = lambda x: np.fmax(x-K, 0)
M = 100
N = 5000

# model parameters
lossFunction = nn.MSELoss()
sizes=[2, 50, 50, 50, 50, 50, 50, 50, 50, 1]
lr = 3e-5
activation = 'relu'

# training parameters
samples = {"pde": 5000, "bc":500, "fc":500}

# sample data generated by finite difference method
X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor = utils.fdm_data(S_range[-1], T, M, N, "500000sample.csv", device)

n_epochs = 20000
pinn_loss_weights = [1, 1, 1]
ipinn_loss_weight = [1, 1, 1]

# Train

In [ ]:
# pinn, min_model, mse_loss_hist, pde_loss_hist, bc_loss_hist, data_loss_hist, weights_hist = utils.network_training(
#         K, r, sigma, T, S_range[-1], S_range, t_range, gs, samples['bc'], samples['fc'], samples['pde'], RNG_key=123,
#         device=device, net='pinn', sizes=sizes, activation=activation, learning_rate=lr, aw_learning_rate=lr, n_epochs=n_epochs, lossFunction=lossFunction, dropout_rate=0, adaptive_rate=None, adaptive_rate_scaler=0, loss_weights=pinn_loss_weights, adaptive_weight=None, X_train_tensor=X_train_tensor, y_train_tensor=y_train_tensor,
#         )

# ipinn, min_model, mse_loss_hist, pde_loss_hist, bc_loss_hist, data_loss_hist, weights_hist = utils.network_training(
#         K, r, sigma, T, S_range[-1], S_range, t_range, gs, samples['bc'], samples['fc'], samples['pde'], RNG_key=123,
#         device=device, net='ipinn', sizes=sizes, activation=activation, learning_rate=lr, aw_learning_rate=lr, n_epochs=n_epochs, lossFunction=lossFunction, dropout_rate =0, adaptive_rate=0.1, adaptive_rate_scaler=10.0, loss_weights=ipinn_loss_weight, adaptive_weight=None, X_train_tensor=X_train_tensor, y_train_tensor=y_train_tensor,
#         )

# awpinn, min_model, mse_loss_hist, pde_loss_hist, bc_loss_hist, data_loss_hist, loss_weights_hist = utils.network_training(
#         K, r, sigma, T, S_range[-1], S_range, t_range, gs, samples['bc'], samples['fc'], samples['pde'], RNG_key=123,
#         device=device, net='pinn', sizes=sizes, activation=activation, learning_rate=lr, aw_learning_rate=0.001, n_epochs=n_epochs, lossFunction=lossFunction, dropout_rate=0, adaptive_rate=None, adaptive_rate_scaler=0, loss_weights=pinn_loss_weights, adaptive_weight=True, X_train_tensor=X_train_tensor, y_train_tensor=y_train_tensor,
#         )

awipinn, min_model, mse_loss_hist, pde_loss_hist, bc_loss_hist, data_loss_hist, loss_weights_hist = utils.network_training(
        K, r, sigma, T, S_range[-1], S_range, t_range, gs, samples['bc'], samples['fc'], samples['pde'], RNG_key=123,
        device=device, net='ipinn', sizes=sizes, activation=activation, learning_rate=lr, aw_learning_rate=0.001, n_epochs=n_epochs, lossFunction=lossFunction, dropout_rate=0, adaptive_rate=0.1, adaptive_rate_scaler=10.0, loss_weights=ipinn_loss_weight, adaptive_weight=True, X_train_tensor=X_train_tensor, y_train_tensor=y_train_tensor,
        )

# Compare Predict and Exact